# Test Loss Functions on Multiple Models

In [ ]:
import sys
sys.path.append('src')
import pandas as pd
import numpy as np
import xgboost as xg
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import yaml
# Local modules
from data_funcs import train_test_split_spacetime
from fmda_models import LM, XGB
from metrics import ros, rmse
import reproducibility

## Read and Split Data

In [ ]:
df = pd.read_pickle("data/rocky_2023_05-09.pkl")
X_train, X_test, y_train, y_test = train_test_split_spacetime(df)

## Setup Models

In [ ]:
def exp_weight(y_train, w=0.1):
    """
    Function to return weight vector of length equal to vector input y. 
    Math Definition: e^(-w*y) Used for weighted loss func.
    Parameters:
    -----------
    y_train : numpy array
        observed data vector. Used on training observations, never on test for forecasts
    w : float, default=0.1
        Column of dataframe to be used for y_train, y_test
    Returns:
    -----------
    Array of length (len(y_train))
    """
    return tf.exp(tf.multiply(-w, y_train))

In [ ]:
def create_exp_function(w):
    def exp_function(y_train):
        return tf.exp(tf.multiply(-w, y_train))
    return exp_function

In [ ]:
fun = create_exp_function(.05)
fun(y_train = np.array([1,2,3]))

In [ ]:
with open('models/params.yaml', 'r') as file:
    params = yaml.safe_load(file)

params

In [ ]:
def get_model(mod):
    if mod == "xgb":
        return XGB(params["xgb"])
    elif mod == "lm":
        return LM()
    else:
        raise ValueError("Invalid model name")

In [ ]:
def models_setup(model_ls, ws=None):
    loss_fucs = ["rss", "exp", "ros"]
    model_types = ["lm", "xgb"]
    # set up return dictionary
    models = {
        'rss' : {
            'w_func' : rmse
        },
        'ros': {
            'w_func' : ros
        }
    } 
    # Using input omega parameter list, add dictionary key for exponential weighting for each omega in list 
    if ws is not None:
        for w in ws:
            assert isinstance(w, float) # Check that given list of floats
            dname = f"exp_{w}" # create name of dictionary key
            models[dname] = {
                'w_func' : create_exp_function(w)
            }

    for lfunc in models:
        for mod in model_ls:
            if mod in model_types:
                models[lfunc][mod]={
                    'model' : get_model(mod)
                }
            else:
                raise ValueError(f"Model `{mod}` not recognized. Accepted models: {model_types}")

    return models

In [ ]:
x = models_setup(["lm", "xgb"], ws=np.linspace(0.01, .3, 3))
x

In [ ]:
x['exp_0.3']["w_func"](np.array([1,2,3]))

In [ ]:
np.exp(-0.3*np.array([1,2,3]))

In [ ]:
def loss_MSE(y_true, y_pred):
    return tf.math.reduce_mean(tf.square(y_true - y_pred))

In [ ]:
# Loss function with weights based on 
# expoential function of amplitude of y_true
def my_MSE_weighted(y_true,y_pred):
    return K.mean(
        tf.multiply(
        tf.exp(tf.multiply(5.0, y_true)),
        tf.square(tf.subtract(y_pred, y_true))
    )
)